In [1]:
import os
import numpy as np
import openslide
import cv2
import pandas as pd
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from skimage import morphology
from tqdm import tqdm

WSI_DIR = "dataset/dataset/HPV"
OUTPUT_PATCHES_DIR = "output_patches/"
EXCEL_FILE = "hpv_labels.xlsx"
PATCH_SIZE = 1024

os.makedirs(OUTPUT_PATCHES_DIR, exist_ok=True)

def load_hpv_labels(excel_path):
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        labels = dict(zip(df['slide_name'], df['hpv_status']))
        return labels
    else:
        return None

def get_tissue_coords(slide, patch_size, downscale_factor=16):
    thumbnail = slide.get_thumbnail((slide.dimensions[0] // downscale_factor, slide.dimensions[1] // downscale_factor))
    grayscale = rgb2gray(np.array(thumbnail))
    thresh = threshold_otsu(grayscale)
    binary_mask = grayscale < thresh
    binary_mask = morphology.remove_small_objects(binary_mask, min_size=500)

    tissue_coords = []
    mask_h, mask_w = binary_mask.shape

    for i in range(0, mask_w - patch_size // downscale_factor, patch_size // downscale_factor):
        for j in range(0, mask_h - patch_size // downscale_factor, patch_size // downscale_factor):
            patch_mask = binary_mask[j:j + patch_size // downscale_factor, i:i + patch_size // downscale_factor]
            if np.sum(patch_mask) > 0.1 * patch_mask.size:
                x = i * downscale_factor
                y = j * downscale_factor
                tissue_coords.append((x, y))

    return tissue_coords

def extract_patches_generator(slide_path, slide_name, labels=None, patch_size=PATCH_SIZE):
    slide = openslide.OpenSlide(slide_path)
    coords = get_tissue_coords(slide, patch_size)

    label = labels.get(slide_name, "Unknown") if labels else None

    for (x, y) in coords:
        patch = slide.read_region((x, y), 0, (patch_size, patch_size)).convert("RGB")
        patch_np = np.array(patch)
        if np.mean(patch_np) < 230:
            yield patch_np, slide_name, x, y, label

def save_patch(patch_np, slide_name, x, y, label, count, save_dir=OUTPUT_PATCHES_DIR):
    target_dir = os.path.join(save_dir, f"HPV_{label}" if label else "All")
    os.makedirs(target_dir, exist_ok=True)
    fname = f"{slide_name}_patch_{count}"
    if label:
        fname += f"_HPV_{label}"
    fname += ".png"
    patch_path = os.path.join(target_dir, fname)
    cv2.imwrite(patch_path, cv2.cvtColor(patch_np, cv2.COLOR_RGB2BGR))

hpv_labels = load_hpv_labels(EXCEL_FILE)
wsi_files = [f for f in os.listdir(WSI_DIR) if f.endswith(".svs")]

for wsi in wsi_files:
    slide_name = os.path.splitext(wsi)[0]
    count = 0
    for patch_np, slide_name, x, y, label in tqdm(
        extract_patches_generator(os.path.join(WSI_DIR, wsi), slide_name, labels=hpv_labels),
        desc=f"Processing {slide_name}"
    ):
        save_patch(patch_np, slide_name, x, y, label, count)
        count += 1

print("✅ WSI Preprocessing Completed!")


Processing 6_27_105333: 854it [02:56,  4.83it/s]
Processing 7_17_203144: 486it [01:43,  4.71it/s]
Processing 7_19_203714: 658it [02:20,  4.70it/s]
Processing 7_1_110654: 572it [02:02,  4.67it/s]
Processing 7_20_204009: 403it [01:23,  4.81it/s]
Processing 7_24_205049: 431it [01:26,  5.00it/s]
Processing 7_26_205537: 626it [01:53,  5.53it/s]
Processing 7_7_200719: 802it [02:41,  4.95it/s]
Processing S-7273-20_1683397172: 604it [01:54,  5.27it/s]
Processing S-7364-20_1683396862: 332it [01:20,  4.15it/s]

✅ WSI Preprocessing Completed!


In [3]:
all_files = os.listdir('output_patches/All')
len(all_files)

5768

In [4]:
import numpy as np
import os
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import shutil

IMAGE_DIR = "output_patches/All"
OUTPUT_DIR = "clustered_data"
# OUTPUT_DIR = "output"
# MODEL_NAME = "resnet"
MODEL_NAME = "efficientnet"
IMAGE_SIZE = (512, 512)

if MODEL_NAME == "resnet":
    base_model = ResNet50(weights="imagenet", include_top=False, pooling='avg')
    preprocess = resnet_preprocess
elif MODEL_NAME == "efficientnet":
    base_model = EfficientNetB0(weights="imagenet", include_top=False, pooling='avg')
    preprocess = efficientnet_preprocess
else:
    raise ValueError("Unsupported model")

image_paths = [os.path.join(IMAGE_DIR, fname) for fname in os.listdir(IMAGE_DIR) if fname.endswith(".png")]
features = []
valid_paths = []

for path in tqdm(image_paths):
    try:
        img = image.load_img(path, target_size=IMAGE_SIZE)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess(img_array)
        feature = base_model.predict(img_array, verbose=0)
        features.append(feature[0])
        valid_paths.append(path)
    except Exception as e:
        print(f"Error processing {path}: {e}")

features = np.array(features)


100%|██████████| 5768/5768 [21:26<00:00,  4.48it/s]


In [5]:
features

array([[-0.05471035, -0.08890644, -0.04283147, ..., -0.05659637,
        -0.0668222 , -0.02000637],
       [-0.1071095 , -0.10361651, -0.00935431, ..., -0.05735889,
        -0.09221352,  0.15958616],
       [-0.08441477, -0.09381215,  0.23475213, ..., -0.04244846,
        -0.08819544,  0.13114902],
       ...,
       [ 0.00948013,  0.37763914,  0.00478341, ..., -0.1009889 ,
        -0.08931585,  0.00726312],
       [-0.07154578, -0.06626318, -0.00125617, ..., -0.07764931,
         0.01026511, -0.09862373],
       [-0.02642258,  0.43536416, -0.12733597, ..., -0.1228595 ,
        -0.04735574, -0.08844389]], dtype=float32)

In [6]:
import pickle

FEATURES_FILE = "features.pkl"

with open(FEATURES_FILE, "wb") as f:
    pickle.dump((features, valid_paths), f)

print(f"Saved {len(features)} feature vectors to '{FEATURES_FILE}'")


Saved 5768 feature vectors to 'features.pkl'


In [7]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

features_normalized = normalize(features, norm='l2')
# features_normalized = normalize(features)

kmeans = KMeans(n_clusters=2, random_state=42)
labels = kmeans.fit_predict(features_normalized)

In [8]:
cluster_dirs = [os.path.join(OUTPUT_DIR, f"cluster_{i}") for i in range(2)]
for d in cluster_dirs:
    os.makedirs(d, exist_ok=True)

for path, label in zip(valid_paths, labels):
    fname = os.path.basename(path)
    shutil.copy(path, os.path.join(cluster_dirs[label], fname))

print("Clustering complete. Images saved to respective cluster directories.")


Clustering complete. Images saved to respective cluster directories.


In [9]:
import joblib

joblib.dump(kmeans, 'classifier_model.pkl')

['classifier_model.pkl']

# Test Flow

In [ ]:
# test_image_path = r"output_patches\All\6_27_105333_patch_0.png"

test_image_path = "test data/human-papilloma-virus-infectio.png"
IMAGE_SIZE = (512, 512)

features = []
try:
    img = image.load_img(test_image_path, target_size=IMAGE_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess(img_array)
    feature = base_model.predict(img_array, verbose=0)
    features.append(feature[0])
    valid_paths.append(path)
except Exception as e:
    print(f"Error processing {path}: {e}")

features = np.array(features)


In [ ]:
import joblib

loaded_model = joblib.load('classifier_model.pkl')

new_data_normalized = normalize(features, norm='l2')
predicted_labels = loaded_model.predict(new_data_normalized)


In [36]:
predicted_labels

array([0], dtype=int32)